In [3]:
import os
import sys

# Sätt base_dir (projektets rotkatalog)
base_dir = os.path.abspath("..")  # En nivå upp från notebooks/
sys.path.append(base_dir)  # Lägg till base_dir i sys.path

# Nu fungerar importerna
from src.objects.optimizer import Optimizer
from src.objects.package import Package
from src.data_processing import load_data
import pandas as pd

lagerstatus = os.path.join(base_dir, "data", "lagerstatus.csv")

# Kontrollera att imports fungerar
print("Imports lyckades!")

Imports lyckades!


In [11]:
log_file = os.path.join(base_dir, "logs", "optimization.log")

# Rensa tidigare loggar
if os.path.exists(log_file):
    os.remove(log_file)

# Skapa testdataset
packages = load_data(lagerstatus)

print(f"Testdata skapad! {len(packages)}")


Testdata skapad! 500


In [12]:
# Originalmetod: Ingen optimering, bara lista paketen
optimizer = Optimizer(packages, max_trucks=10, max_capacity=800)
optimizer.distribute_packages()
optimizer.display_results()


Total remaining packages in storage: 0

--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 799.7699999999999, Packages: 170)
Truck(Truck_2, Total Weight: 798.9799999999998, Packages: 73)
Truck(Truck_3, Total Weight: 797.9699999999997, Packages: 64)
Truck(Truck_4, Total Weight: 797.48, Packages: 58)
Truck(Truck_5, Total Weight: 796.43, Packages: 61)
Truck(Truck_6, Total Weight: 798.1600000000001, Packages: 61)
Truck(Truck_7, Total Weight: 229.29000000000002, Packages: 13)

Totalt antal paket kvar i lager: 0
Total Förtjänst (levererade paket): 913.0
Totala Straffavgifter: -1769.0
Använda bilar: 7


In [13]:
# Greedy Optimering
optimizer_greedy = Optimizer(packages, max_trucks=10, max_capacity=800, log_file=log_file)
optimizer_greedy.greedy_optimization()
optimizer_greedy.display_results()



--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 799.7699999999999, Packages: 170)
Truck(Truck_2, Total Weight: 798.9799999999998, Packages: 73)
Truck(Truck_3, Total Weight: 797.9699999999997, Packages: 64)
Truck(Truck_4, Total Weight: 797.48, Packages: 58)
Truck(Truck_5, Total Weight: 796.43, Packages: 61)
Truck(Truck_6, Total Weight: 798.1600000000001, Packages: 61)
Truck(Truck_7, Total Weight: 229.29000000000002, Packages: 13)

Totalt antal paket kvar i lager: 0
Total Förtjänst (levererade paket): 913.0
Totala Straffavgifter: -1769.0
Använda bilar: 7


In [14]:
# Genetisk Algoritm
optimizer_genetic = Optimizer(packages, max_trucks=10, max_capacity=800, log_file=log_file)
optimizer_genetic.genetic_optimization(population_size=10, generations=50, mutation_rate=0.1)
optimizer_genetic.display_results()



--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 525.6800000000001, Packages: 50)
Truck(Truck_2, Total Weight: 465.7599999999999, Packages: 50)
Truck(Truck_3, Total Weight: 477.39000000000004, Packages: 50)
Truck(Truck_4, Total Weight: 455.18999999999994, Packages: 50)
Truck(Truck_5, Total Weight: 522.9300000000002, Packages: 50)
Truck(Truck_6, Total Weight: 482.04999999999995, Packages: 50)
Truck(Truck_7, Total Weight: 478.42999999999995, Packages: 50)
Truck(Truck_8, Total Weight: 460.60999999999996, Packages: 50)
Truck(Truck_9, Total Weight: 572.43, Packages: 50)
Truck(Truck_10, Total Weight: 477.5199999999998, Packages: 50)

Totalt antal paket kvar i lager: 0
Total Förtjänst (levererade paket): 1252.0
Totala Straffavgifter: -1491.0
Använda bilar: 10


In [32]:
# Greedy + Genetisk Algoritm
optimizer_combined = Optimizer(packages, max_trucks=10, max_capacity=800, log_file=log_file)
optimizer_combined.greedy_optimization()  # Börja med greedy
optimizer_combined.genetic_optimization(population_size=10, generations=50, mutation_rate=0.1)  # Förbättra
optimizer_combined.display_results()



--- Resultat för Optimering ---
Truck(Truck_1, Total Weight: 477.48, Packages: 50)
Truck(Truck_2, Total Weight: 565.81, Packages: 50)
Truck(Truck_3, Total Weight: 534.9, Packages: 50)
Truck(Truck_4, Total Weight: 453.81999999999994, Packages: 50)
Truck(Truck_5, Total Weight: 509.3000000000002, Packages: 50)
Truck(Truck_6, Total Weight: 447.78000000000003, Packages: 50)
Truck(Truck_7, Total Weight: 496.85999999999984, Packages: 50)
Truck(Truck_8, Total Weight: 516.4100000000001, Packages: 50)
Truck(Truck_9, Total Weight: 515.4, Packages: 50)
Truck(Truck_10, Total Weight: 503.37, Packages: 50)

Totalt antal paket kvar i lager: 0
Total Förtjänst (levererade paket): 996.0
Totala Straffavgifter: -1732.0
Använda bilar: 10


In [33]:
# Läs loggfilen
log_data = []
with open(log_file, "r") as log:
    for line in log:
        if "Iteration" in line:
            try:
                parts = line.strip().split(", ")
                iteration = int(parts[0].split(": ")[1])
                fitness_score = parts[1].split(": ")[1]
                fitness_score = float(fitness_score) if fitness_score != "N/A" else None
                score = float(parts[2].split(": ")[1])
                total_profit = float(parts[3].split(": ")[1])
                total_penalty = float(parts[4].split(": ")[1])
                remaining_packages = int(parts[5].split(": ")[1])
                used_trucks = int(parts[6].split(": ")[1])
                log_data.append((iteration, fitness_score, score, total_profit, total_penalty, remaining_packages, used_trucks))
            except (IndexError, ValueError) as e:
                print(f"Fel vid bearbetning av rad: {line}, fel: {e}")

# Skapa DataFrame för analys
if not log_data:
    print("Inga data kunde läsas från loggfilen.")
else:
    df_logs = pd.DataFrame(
        log_data,
        columns=["Iteration", "Fitness Score", "Score", "Total Profit", "Total Penalty", "Remaining Packages", "Used Trucks"]
    )

    # Visualisera förbättringar
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 6))
    plt.plot(df_logs["Iteration"], df_logs["Score"], label="Score (Net Profit)")
    plt.plot(df_logs["Iteration"], df_logs["Fitness Score"], label="Fitness Score", linestyle="--")
    plt.xlabel("Iteration")
    plt.ylabel("Value")
    plt.title("Förbättringar med Optimeringsalgoritmer")
    plt.legend()
    plt.grid()
    plt.show()


Fel vid bearbetning av rad: Iteration: 0, Total Förtjänst: 913.0, Total Straffavgifter: -1769.0, Kvarvarande paket: 0, Använda bilar: 7
, fel: list index out of range
Fel vid bearbetning av rad: Iteration: 1, Total Förtjänst: 0, Total Straffavgifter: 0, Kvarvarande paket: 0, Använda bilar: 0
, fel: list index out of range
Fel vid bearbetning av rad: Iteration: 2, Total Förtjänst: 0, Total Straffavgifter: 0, Kvarvarande paket: 0, Använda bilar: 0
, fel: list index out of range
Fel vid bearbetning av rad: Iteration: 3, Total Förtjänst: 0, Total Straffavgifter: 0, Kvarvarande paket: 0, Använda bilar: 0
, fel: list index out of range
Fel vid bearbetning av rad: Iteration: 4, Total Förtjänst: 0, Total Straffavgifter: 0, Kvarvarande paket: 0, Använda bilar: 0
, fel: list index out of range
Fel vid bearbetning av rad: Iteration: 5, Total Förtjänst: 0, Total Straffavgifter: 0, Kvarvarande paket: 0, Använda bilar: 0
, fel: list index out of range
Fel vid bearbetning av rad: Iteration: 6, Total 

In [10]:
import pandas as pd
# Load the CSV file into a DataFrame
df_lagerstatus = pd.read_csv(lagerstatus)

# Calculate the sum of 'fortjanst'
total_fortjanst = df_lagerstatus['fortjanst'].sum()

# Calculate the sum of 'deadline'
total_deadline = df_lagerstatus['deadline'].sum()

# Calculate penalty for negative deadlines
penalty = df_lagerstatus[df_lagerstatus['deadline'] < 0]['deadline'].apply(lambda x: -x**2).sum()
total_deadline += penalty

total = total_fortjanst - total_deadline

print(f"Total Fortjanst: {total_fortjanst}")
print(f"Total penalty : {total_deadline}")
print(f"total : {total}")

Total Fortjanst: 2682
Total penalty : -446
total : 3128
